In [ ]:
!pip install -q datasets transformers accelerate

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image

In [ ]:
data = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/summer24/lichen classification/Train_DATA")

Resolving data files:   0%|          | 0/683 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 683
    })
})

In [ ]:
labels = data["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'fruticose'

In [ ]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    RandomAffine,
    RandomRotation,
    Resize,
    ToTensor,
)

In [ ]:
splits = data["train"].train_test_split(test_size=0.2)
train = splits['train']
val_ds = splits['test']

In [ ]:
# transforms_aug=Compose([
#     transforms.RandomAffine(degrees=(-5, 5), translate=(0.33, 0.33), scale=(1.0, 1.5)),
#     transforms.RandomChoice([
#         transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomVerticalFlip(p=0.5)])
# ])

In [ ]:
# augmented_dataset = train.map(
#     lambda x: {"image": transforms_aug(x["image"])},
#     batched=False  # Process batches for efficiency
# )

In [ ]:
# from datasets import concatenate_datasets

# train_ds = concatenate_datasets([augmented_dataset, train])

In [ ]:
transform_rotate =transforms.RandomRotation(degrees=(-5, 5))
transform_Hflip = transforms.RandomHorizontalFlip(p=1)
transform_Vflip = transforms.RandomVerticalFlip(p=1)
transform_affine = transforms.RandomAffine(degrees=0, translate=(0.20, 0.20), scale=(1.0, 1.5))

In [ ]:
from torchvision.transforms import functional as F

def equalize_image(image):
    return F.equalize(image)

def apply_transform(example, transform):
    example['image'] = transform(example['image'])
    return example

train_Hflip = train.map(lambda x: apply_transform(x, transform_Hflip))
train_Vflip = train.map(lambda x: apply_transform(x, transform_Vflip))
train_affine = train.map(lambda x: apply_transform(x, transform_affine))
train_rotate = train.map(lambda x: apply_transform(x, transform_rotate))


Map:   0%|          | 0/546 [00:00<?, ? examples/s]

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

In [ ]:
from datasets import concatenate_datasets

train_ds = concatenate_datasets([train_Hflip,train_Vflip,train_affine,train_rotate,train])

In [ ]:
train_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 2730
})

In [ ]:
val_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 137
})

In [ ]:
train_ds = train_ds.shuffle(seed=42)
train_ds = train_ds.flatten_indices()

Flattening the indices:   0%|          | 0/2730 [00:00<?, ? examples/s]

In [ ]:
def pre_transforms(examples):
    examples["pixel_values"] = [image.convert("RGB").resize((224,224)) for image in examples["image"]]
    return examples

T = train_ds.map(pre_transforms, remove_columns=["image"], batched=True)
V= val_ds.map(pre_transforms, remove_columns=["image"], batched=True)

Map:   0%|          | 0/2730 [00:00<?, ? examples/s]

Map:   0%|          | 0/137 [00:00<?, ? examples/s]

In [ ]:
T

Dataset({
    features: ['label', 'pixel_values'],
    num_rows: 2730
})

In [ ]:
from PIL import Image
import cv2

def rgb_to_hsv(image_np):
    """ Convert RGB numpy array to HSV. """
    return cv2.cvtColor(image_np, cv2.COLOR_RGB2HSV)

def process_image(example):

    image = Image.fromarray(np.array(example['pixel_values'], dtype=np.uint8))
    image_np = np.array(image)

    image_hsv_np = rgb_to_hsv(image_np)

    example['pixel_values'] = image_hsv_np

    return example

In [ ]:
T = T.map(process_image)
V = V.map(process_image)

Map:   0%|          | 0/2730 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/features/image.py:339: UserWarning: Downcasting array dtype int64 to uint8 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


Map:   0%|          | 0/137 [00:00<?, ? examples/s]

In [ ]:
# Check the type and shape of pixel_values
example = T[0]
pixel_values = example['pixel_values']
print(type(pixel_values))
print(pixel_values.shape if isinstance(pixel_values, np.ndarray) else "Not a NumPy array")


<class 'PIL.PngImagePlugin.PngImageFile'>
Not a NumPy array


In [ ]:
T_np=np.array(T['pixel_values'])
V_np=np.array(V['pixel_values'])

In [ ]:
from skimage.feature import local_binary_pattern

def lbp_features(image):
    feat=[]
    for i in range(3):
      g_img=image[:,:,i]
      lbp_img=local_binary_pattern(image=g_img, P=8, R=1)
      lbp_img = np.nan_to_num(lbp_img)
      lbp_hist, _ = np.histogram(lbp_img.ravel(), bins=9, range=(0, 9), density=True)
      feat.append(lbp_hist)

    features=np.concatenate(feat)
    return features


In [ ]:
T_features=np.zeros((len(T_np),27))
V_features=np.zeros((len(V_np),27))

for i in range(len(T_np)):
  T_features[i]=lbp_features(T_np[i])

for i in range(len(V_np)):
  V_features[i]=lbp_features(V_np[i])

/usr/local/lib/python3.10/dist-packages/numpy/lib/histograms.py:885: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


In [ ]:
type(T['label'])

list

In [ ]:
T_features

array([[0.40744239, 0.10615791, 0.0164337 , ..., 0.07478428, 0.17372963,
        0.01764142],
       [0.25632911, 0.11663653, 0.00316456, ..., 0.07445336, 0.16191531,
        0.01882092],
       [0.22790566, 0.13554987, 0.00341006, ..., 0.06919155, 0.14522942,
        0.02388929],
       ...,
       [0.27097768, 0.09815242, 0.00269438, ..., 0.07265657, 0.18706508,
        0.01289398],
       [0.38512616, 0.09030544, 0.00752545, ..., 0.08577667, 0.16297567,
        0.0102932 ],
       [0.39526731, 0.11685656, 0.01110137, ..., 0.04752113, 0.06636966,
        0.02753027]])

In [ ]:
from sklearn import svm
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean') # Replace NaNs with mean of column
X= imputer.fit_transform(T_features)

y = T['label']
clf = svm.SVC(C=5,gamma=3.6)
clf.fit(X, y)

SVC(C=5, gamma=3.6)

In [ ]:
X_test=imputer.fit_transform(V_features)
y_test = V['label']
print(clf.score(X_test, y_test))

0.635036496350365


In [ ]:
print(clf.score(X, y))

0.6725274725274726


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rf= RandomForestClassifier(n_estimators=3)
clf_rf.fit(X, y)

RandomForestClassifier(n_estimators=3)

In [ ]:
print(clf_rf.score(X_test, y_test))

0.4744525547445255


In [ ]:
print(clf_rf.score(X, y))

0.9380952380952381


In [ ]:
from sklearn.linear_model import LogisticRegression

clf_LR = LogisticRegression(C=10).fit(X, y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
print(clf_LR.score(X_test, y_test))

0.5255474452554745


In [ ]:
print(clf_LR.score(X, y))

0.5347985347985348
